## Imports and Initialise

In [1]:
import os
import re
import csv
import json
import random
from openai import OpenAI
from dotenv import load_dotenv
from llm_generate import get_all_paths, get_generate_prompt, process_mwo_response
from llm_prompt import initialise_prompts, check_similarity
from llm_paraphrase import paraphrase_mwo

load_dotenv()
api_key = os.getenv('API_KEY')
client = OpenAI(api_key=api_key)
data, _ = get_all_paths(valid=True)

c:\Users\allis\miniconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


49	paths in object_property_paths
391	paths in process_agent_paths
269	paths in process_patient_paths
1208	paths in state_patient_paths
3	paths in object_property_state_paths
0	paths in object_process_state_paths
607	paths in state_agent_activity_paths
37	paths in state_agent_patient_paths
1231	paths in process_agent_patient_paths
Total number of paths: 3795


## Functions

In [2]:
def get_1fewshot_message(prompt_variations):
    """ Get fewshot message given fewshot csv file for single-example prompt """
    message = [{"role": "system", "content": "You are a technician recording maintenance work orders."}]
    with open("fewshot_messages/fewshot_one.csv", encoding='utf-8') as f:
        data = csv.reader(f)
        next(data) # Ignore header
        for row in data:
            object_name = row[0]
            event_name = f"{row[1]} {row[2]}".strip()
            prompt = get_generate_prompt(prompt_variations, object_name, event_name)
            user = {"role": "user", "content": prompt}
            assistant = {"role": "assistant", "content": row[3]}
            message.append(user)
            message.append(assistant)
    return message

def get_5fewshot_message(prompt_variations):
    """ Get fewshot message given fewshot csv file for 5-example prompt """
    message = [{"role": "system", "content": "You are a technician recording maintenance work orders."}]
    with open("fewshot_messages/fewshot_generate.csv", encoding='utf-8') as f:
        data = csv.reader(f)
        next(data) # Ignore header
        for row in data:
            object_name = row[0]
            event_name = f"{row[1]} {row[2]}".strip()
            prompt = get_generate_prompt(prompt_variations, object_name, event_name)
            user = {"role": "user", "content": prompt}
            example = f"1. {row[4]}\n2. {row[5]}\n3. {row[6]}\n4. {row[7]}\n5. {row[8]}"
            assistant = {"role": "assistant", "content": example}
            message.append(user)
            message.append(assistant)
    return message

# Print some fewshot examples from MaintIE gold dataset
def print_examples(object, event, helper=None):
    """ Print some fewshot examples from the gold dataset """
    data = []
    with open("data/MaintIE/gold_release.json", encoding='utf-8') as f:
        gold = json.load(f)
        for d in gold:
            text = d['text'].replace("<id> ", "").replace(" <id>", "")
            data.append(text)
    for sentence in data:
        event_exists = re.search(rf'\b{event}\b', sentence)
        helper_exists = re.search(rf'\b{helper}\b', sentence) if helper else None
        if object in sentence and event_exists and helper_exists:
            print(f"{object},{event},{helper},{sentence}")
            return True
        elif object in sentence and event_exists:
            print(f"{object},{event},{sentence}")
            return True
    return False

# Process single response from the LLM
def process_single_response(response):
    """ Process single response from the LLM """
    response = response.lower()                     # Case folding
    response = re.sub(r'[^\w\s]', ' ', response)    # Remove punctuation
    response = re.sub(r"\s+", " ", response)        # Remove extra spaces
    return response

# write to outlog text file
def write_to_outlog(title, data):
    """ Write data to log text file """
    with open("mwo_sentences/path_diversity.txt", "a", encoding='utf-8') as f:
        f.write("========================================\n")
        f.write(f"{title}\n")
        f.write(f"- Number of unique: {len(data)}\n")
        for d in data:
            f.write(f"{d}\n")
        f.write("========================================\n")

In [4]:
# Print some examples from the gold dataset
successful_calls = 0
while successful_calls < 5:
    current = random.choice(data)
    if 'helper_name' in current:
        if print_examples(current['object_name'], current['event_name'], current['helper_name']):
            successful_calls += 1
    else:
        if print_examples(current['object_name'], current['event_name']):
            successful_calls += 1

steering column,movement,movement on steering column
aftercooler,error,aftercooler temperature error
brake,pressure fault,swing brake pressure fault
air conditioner,needs,boost,air conditioner needs a boost not cold
track frame,crack,- repair right hand track frame crack


## Same prompt VS Variant prompts

In [4]:
path = {
        "object_name": "park brake",
        "event_name": "alarm fault",
        "valid": True,
        "alternate": False
    }
prompt_variations = initialise_prompts(client, num_variants=5, num_examples=1)
base_prompts, limit_words, limit_count = prompt_variations
for b in base_prompts:
    print(b)
for w in limit_words:
    print(w)
for c in limit_count:
    print(c)

c:\Users\allis\miniconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Create a Maintenance Work Order (MWO) sentence detailing the equipment and the undesirable event.
Write a sentence for a Maintenance Work Order (MWO) referring to the equipment and the undesirable event.
Compose a sentence for a Maintenance Work Order (MWO) that specifies the equipment and the undesirable event.
Draft a sentence for a Maintenance Work Order (MWO) that outlines the equipment and the undesirable event.
Formulate a sentence for a Maintenance Work Order (MWO) that describes the equipment and the undesirable event.
Avoid verbosity and minimize stop words.
Don't use verbosity; keep stop words to a minimum.
Refrain from verbosity and limit stop words.
Eliminate verbosity and reduce stop words.
Skip verbosity and cut down on stop words.
The sentence can only be 8 words long.
A sentence must have a maximum of 8 words.
The sentence can contain up to 8 words.
Ensure the sentence is 8 words or fewer.
The sentence should not exceed 8 words.


In [5]:
# Same prompt
def same_prompt():
    outputs = []
    base_prompt = "Generate a Maintenance Work Order (MWO) sentence describing the following equipment and undesirable event."
    limit_words = "Avoid verbosity and use minimal stop words."
    limit_count = "The sentence can have a maximum of 8 words."
    prompt = f"{base_prompt}\nEquipment: {path['object_name']}\nUndesirable Event: {path['event_name']}"
    prompt += f"You must use all terms given above and do not add new information.\n{limit_words}\n{limit_count}"
    fewshot = get_1fewshot_message(([base_prompt], [limit_words], [limit_count]))
    message = fewshot + [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=message,
                    temperature=0.9,
                    top_p=0.9,
                    n=5
                )

    for choice in response.choices:
        output = choice.message.content
        output = process_single_response(output) 
        outputs.append(output)
    return outputs

In [6]:
# Variant prompts
def variant_prompt():
    outputs = []
    fewshot = get_1fewshot_message(prompt_variations)
    for _ in range(5):
        prompt = get_generate_prompt(prompt_variations, path['object_name'], path['event_name'])
        message = fewshot + [{"role": "user", "content": prompt}]
        response = client.chat.completions.create(
                        model="gpt-4o-mini",
                        messages=message,
                        temperature=0.9,
                        top_p=0.9,
                        n=1
                )

        for choice in response.choices:
            output = choice.message.content
            output = process_single_response(output)
            outputs.append(output)
    return outputs

In [7]:
# Demo
same = same_prompt()
variant = variant_prompt()

print ("GPT given same prompt:")
for s in same:
    print(f"- {s}")
print ("GPT given variant prompts:")
for v in variant:
    print(f"- {v}")

GPT given same prompt:
- park brake alarm fault detected
- park brake alarm fault detected
- park brake alarm fault
- park brake alarm fault
- park brake alarm fault
GPT given variant prompts:
- park brake has alarm fault
- park brake has alarm fault
- park brake alarm fault detected
- park brake alarm fault detected
- park brake alarm fault


In [17]:
# Compare diversity over long run
same = []
variant = []
for _ in range(20):
    same.extend(same_prompt())
    variant.extend(variant_prompt())
print (f"Number of unique responses for same prompt: {len(set(same))}")
print (f"Number of unique responses for variant prompts: {len(set(variant))}")
write_to_outlog("OUTPUT FOR SAME PROMPT", list(set(same)))
write_to_outlog("OUTPUT FOR VARIANT PROMPTS", list(set(variant)))
total = same + variant
print (f"Total number of unique responses: {len(set(total))}")

Number of unique responses for same prompt: 3
Number of unique responses for variant prompts: 3
Total number of unique responses: 3


## N completions VS N sentences in ONE completion

In [9]:
# N sentences in ONE completion
def n_sentences():
    outputs = []
    base_prompt = "Generate 5 different Maintenance Work Order (MWO) sentences describing the following equipment and undesirable event."
    limit_words = "Avoid verbosity and use minimal stop words."
    limit_count = "Each sentence can have a maximum of 8 words."
    prompt = f"{base_prompt}\nEquipment: {path['object_name']}\nUndesirable Event: {path['event_name']}"
    prompt += f"You must use all terms given above and do not add new information.\n{limit_words}\n{limit_count}"
    fewshot = get_5fewshot_message(([base_prompt], [limit_words], [limit_count]))
    message = fewshot + [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=message,
                    temperature=0.9,
                    top_p=0.9,
                    n=1
                )

    for choice in response.choices:
        output = choice.message.content
        output = process_mwo_response(output)
        for sentence in output:
            outputs.append(sentence)
    return outputs

In [10]:
# Demo
n_completion = same_prompt()
n_sentence = n_sentences()

print ("Output in N completions:")
for c in n_completion:
    print(f"- {c}")
print ("Output within 1 completion:")
for s in n_sentence:
    print(f"- {s}")

Output in N completions:
- park brake alarm fault detected
- park brake alarm fault
- park brake alarm fault
- park brake alarm fault
- park brake alarm fault detected
Output within 1 completion:
- park brake alarm fault
- alarm fault in park brake
- park brake has alarm fault
- alarm fault found in park brake
- park brake showing alarm fault


In [11]:
# Compare diversity over long run
n_completion = []
n_sentence = []
for _ in range(20):
    n_completion.extend(same_prompt())
    n_sentence.extend(n_sentences())
print (f"Number of unique responses for N completions: {len(set(n_completion))}")
print (f"Number of unique responses for ONE completion: {len(set(n_sentence))}")
write_to_outlog("OUTPUT IN N COMPLETIONS", list(set(n_completion)))
write_to_outlog("OUTPUT IN 1 COMPLETION", list(set(n_sentence)))
total_responses = n_completion + n_sentence
print (f"Total number of unique responses: {len(set(total_responses))}")

Number of unique responses for N completions: 2
Number of unique responses for ONE completion: 15
Total number of unique responses: 15


## Paraphrase VS Generation

In [12]:
# Paraphrase
def paraphrase():
    outputs = []
    base_prompt = "Generate a Maintenance Work Order (MWO) sentence describing the following equipment and undesirable event."
    limit_words = "Avoid verbosity and use minimal stop words."
    limit_count = "Each sentence can have a maximum of 8 words."
    prompt = f"{base_prompt}\nEquipment: {path['object_name']}\nUndesirable Event: {path['event_name']}"
    prompt += f"You must use all terms given above and do not add new information.\n{limit_words}\n{limit_count}"
    fewshot = get_1fewshot_message(([base_prompt], [limit_words], [limit_count]))
    message = fewshot + [{"role": "user", "content": prompt}]

    response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=message,
                    temperature=0.9,
                    top_p=0.9,
                    n=1
            )

    for choice in response.choices:
        output = choice.message.content
        output = process_single_response(output)
        outputs.append(output)

    keywords = [path['object_name'], path['event_name']]
    paraphrases = paraphrase_mwo(client, output, keywords, 5)
    outputs.extend(paraphrases)
    outputs = list(set(outputs)) # Remove duplicates
    return outputs[:5] # Only return 5 outputs

In [15]:
# Demo
paraphrases = paraphrase()
n_sentence = n_sentences()

print ("Output for Paraphrasing:")
for p in paraphrases:
    print(f"- {p}")
print ("Output for Generation:")
for s in n_sentence:
    print(f"- {s}")

Output for Paraphrasing:
- detected alarm fault in park brake
- park brake alarm fault detected
- park brake has alarm fault
- alarm fault detected in park brake
- alarm fault found in park brake
Output for Generation:
- park brake alarm fault
- alarm fault in park brake
- park brake has alarm fault
- park brake alarm is faulty
- faulty alarm in park brake


In [16]:
# Compare diversity over long run
paraphrases = []
n_sentence = []
for _ in range(20):
    paraphrases.extend(paraphrase())
    n_sentence.extend(n_sentences())
print (f"Number of unique responses for paraphrasing: {len(set(paraphrases))}")
print (f"Number of unique responses for generation: {len(set(n_sentence))}")
write_to_outlog("OUTPUT FOR PARAPHRASING", list(set(paraphrases)))
write_to_outlog("OUTPUT FOR GENERATION", list(set(n_sentence)))
total_responses = paraphrases + n_sentence
print (f"Total unique responses: {len(set(total_responses))}")

Number of unique responses for paraphrasing: 25
Number of unique responses for generation: 16
Total unique responses: 32


## Other Paraphraser Models

In [34]:
import torch
import warnings
warnings.filterwarnings("ignore")

def format_data():
    finetune_data = []
    with open('fewshot_messages/fewshot.csv', 'r', encoding='utf-8') as f:
        data = csv.reader(f)
        next(data) # Ignore header
        for row in data:
            if len(row) > 4:
                original = row[3]
                example = [row[4], row[5], row[6], row[7]]
                for e in example:
                    temp = {
                        "source": original, 
                        "target": e
                    }
                    finetune_data.append(temp)
                    
    finetune_train = finetune_data[:int(0.8*len(finetune_data))]
    finetune_val = finetune_data[int(0.8*len(finetune_data)):]
    with open('fewshot_messages/train.json', 'w', encoding='utf-8') as f:
        json.dump(finetune_train, f, indent=4)
    with open('fewshot_messages/val.json', 'w', encoding='utf-8') as f:
        json.dump(finetune_val, f, indent=4)

# Transformer model for paraphrasing
def model_paraphrase(model, tokenizer, sentence):
    input_ids = tokenizer(
        f'paraphrase: {sentence}',
        return_tensors="pt", padding="longest",
        max_length=25, truncation=True).input_ids.to(device)
    outputs = model.generate(
        input_ids, temperature=0.7, repetition_penalty=10.0,
        num_return_sequences=5, no_repeat_ngram_size=2,
        num_beams=5, num_beam_groups=5,
        max_length=25, diversity_penalty=3.0
    )
    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return res

def paraphrase_sentences(tokenizer, model, sentences):
    outputs = {}
    for sentence in sentences:
        batch = tokenizer([sentence],truncation=True,padding='longest',max_length=25, return_tensors="pt").to(device)
        paraphrased = model.generate(**batch, temperature=0.7, repetition_penalty=10.0,
                                        num_return_sequences=5, no_repeat_ngram_size=2,
                                        num_beams=5, num_beam_groups=5,
                                        max_length=25, diversity_penalty=3.0
                                    )
        output = tokenizer.batch_decode(paraphrased, skip_special_tokens=True)
        outputs[sentence] = output
    return outputs

format_data()

sentences = ["park brake alarm fault detected",
             "alarm fault in park brake system",
             "park brake has alarm fault",
             "park brake system shows alarm fault",
             "alarm fault present in park brake",
             "The boy is walking happily on the street"]    

## BART Paraphrase Model

In [36]:
# https://huggingface.co/eugenesiow/bart-paraphrase
from transformers import BartForConditionalGeneration, BartTokenizer

bart_model = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase')
bart_tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
bart_model = bart_model.to(device)

bart_outputs = paraphrase_sentences(bart_tokenizer, bart_model, sentences)

print("BART Outputs:")
for sentence, outputs in bart_outputs.items():
    print(f"{sentence}:")
    for output in outputs:
        print(f"- {output}")

BART Outputs:
park brake alarm fault detected:
- Park brake alarm fault detected
- The alarm fault was detected in the park brake.
- park Brake alarm fault detected.
- Brake brake alarm Fault detected
- Park brake alarm Fault detect
alarm fault in park brake system:
- Alarm Fault in park brake system?
- What is the cause of an alarm fault in a park brake system?
- The alarm fault is in park brake system.
- In park brake system, alarm fault is not fixed.
- System Fault in park brake system?
park brake has alarm fault:
- Park brake has alarm fault. How can it be fixed?
- The park brake has alarm fault. How can it be fixed?
- What is the cause of a brake alarm fault?
- Park brakes have an alarm fault. What should I do to fix it?
- park brake has alarm faults, it is not working.
park brake system shows alarm fault:
- Park brake system shows alarm fault.
- The park brake system shows alarm fault.
- Park brakes show alarm fault. How can this be fixed?
- A parking brake system shows an alarm 

## T5 Paraphrase Model

In [32]:
# https://huggingface.co/ramsrigouthamg/t5_sentence_paraphraser
from transformers import T5ForConditionalGeneration,T5Tokenizer

t5_sent_model = T5ForConditionalGeneration.from_pretrained("ramsrigouthamg/t5_sentence_paraphraser")
t5_sent_tokenizer = T5Tokenizer.from_pretrained("ramsrigouthamg/t5_sentence_paraphraser")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
t5_sent_model = t5_sent_model.to(device)

for sentence in sentences:
    outputs = model_paraphrase(t5_sent_model, t5_sent_tokenizer, sentence)
    print(f"{sentence}:")
    for output in outputs:
        print(f"- {output.removeprefix('paraphrasedoutput: ')}")

park brake alarm fault detected:
- A park brake warning error has been found.
- An accident has been reported with the park brake alert.
- There has been a problem with the park brake alert system.
- A parking brake alarm fault was detected.
- The park brake alert system is running fine, but it has been detected.
alarm fault in park brake system:
- An alarm is raised in the park brake system.
- An alarm system in the park brake system is failing.
- There is an error in the park brake system that causes this alert.
- an error occurred in the park brake system, causing an alarm.
- In the park brake system, an alarm clock malfunctions.
park brake has alarm fault:
- The park brake has an alarm clock fault.
- The park brake has an error, according to the driver.
- A security issue with the park brake has been fixed.
- An alarm is triggered by the park brake.
- If the park brake has an error, it is possible to fix it.
park brake system shows alarm fault:
- The park brake system is showing an

## ChatGPT Paraphrase T5

In [13]:
# https://huggingface.co/humarin/chatgpt_paraphraser_on_T5_base
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

chat_tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")
chat_model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
chat_model = chat_model.to(device)

for sentence in sentences:
    outputs = model_paraphrase(chat_model, chat_tokenizer, sentence)
    print(f"{sentence}:")
    for output in outputs:
        print(f"- {output}")

park brake alarm fault detected:
- A fault in the park brake alarm has been identified.
- Detection of a fault in the park brake alarm system
- The park brake alarm has been deemed defective.
- An error has been identified in the park brake alarm's functionality.
- Park brake alarm faulty, please.
alarm fault in park brake system:
- An alarm failure has been detected in the park brake system.
- The park brake system has been deemed to be defective due to an
- A faulty alarm in the parking brake system has been detected.
- There is an alarm failure in the park brake system.
- Park brakes issue alarm
park brake has alarm fault:
- The park brake is experiencing an alarm failure.
- Alarm jamming on park brake
- Park brake with alarm faulty.
- An alarm is malfunctioning on the park brake.
- Alarm caliper failing on park brake.
park brake system shows alarm fault:
- The alarm is malfunctioning on the park brake system.
- Alarm triggered by park brake system.
- Alarm failure occurs in the par

## Parrot Paraphrase Model T5

In [6]:
# https://huggingface.co/prithivida/parrot_paraphraser_on_T5
from parrot import Parrot

parrot = Parrot(model_tag="prithivida/parrot_paraphraser_on_T5", use_gpu=False)

for sentence in sentences:
    print(sentence)
    paraphrases = parrot.augment(input_phrase=sentence,
                                diversity_ranker="levenshtein",
                                do_diverse=True, 
                                max_return_phrases=5, 
                                max_length=8)
    if paraphrases:
        for p in paraphrases:
            print(f"- {p}")
    else:
        print("No paraphrases found.")

park brake alarm fault detected
- ('park brake alarm fault detected', 0)
alarm fault in park brake system
- ('warning of a fault in the brake system', 17)
park brake has alarm fault
No paraphrases found.
park brake system shows alarm fault
- ('the park brakes system shows an alarm fault', 13)
alarm fault present in park brake
- ('warning of a fault in the park brake', 22)
The boy is walking happily on the street
- ('a happy boy walks along the streets', 26)
- ('he walks happily on the street', 23)
- ('the boy walks happily in the street', 19)
- ('the boy walks happily on the street', 18)
- ('the boy is walking happily on the street', 12)


## PEGASUS fine-tuned for Paraphrasing

In [15]:
# https://huggingface.co/tuner007/pegasus_paraphrase
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

pegasus_model = PegasusForConditionalGeneration.from_pretrained("tuner007/pegasus_paraphrase")
pegasus_tokenizer = PegasusTokenizer.from_pretrained("tuner007/pegasus_paraphrase")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pegasus_model = pegasus_model.to(device)

for sentence in sentences:
    outputs = model_paraphrase(pegasus_model, pegasus_tokenizer, sentence)
    print(f"{sentence}:")
    for output in outputs:
        print(f"- {output}")

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


park brake alarm fault detected:
- The park brake alarm fault was detected.
- Park brake alarm fault detected.
- A fault with the park brake alarm has been detected.
- There is a park brake alarm fault
- Park brakes fault detected.
alarm fault in park brake system:
- There is an alarm fault in the park brake system.
- The alarm fault in the park brake system.
- Park brake system has an alarm fault.
- It was a fault in the park brake system.
- A fault in the park brake system was cited.
park brake has alarm fault:
- The park brake has an alarm fault.
- Park brake has an alarm fault.
- There is an alarm fault with the park brake.
- The alarm fault is on the park brake.
- A park brake has a problem
park brake system shows alarm fault:
- The park brake system has an alarm fault.
- Park brake system shows alarm fault.
- There is an alarm fault in the park brake system.
- A park brakes alarm shows a fault.
- The Park Brake System shows alarm fault
alarm fault present in park brake:
- There i

## Overall Generation Process
For generating sentences for ONE path, we can use the following process:
1. Use multiple **prompt variations**
2. Generate **multiple sentences** for each completion using a random prompt
3. Generate **paraphrases** for each generated sentence

In [24]:
# Generate 100 random sentences using diverse
base_prompts, limit_words, limit_count = initialise_prompts(client, num_variants=5, num_examples=5)
paths_sample = random.sample(data, 100)
sentences_sample = []

# Generate sentences for paths
for i, path in enumerate(paths_sample):
    # Variant prompt
    prompt = get_generate_prompt(base_prompts, limit_words, limit_count, path['object_name'], path['event_name'])
    keywords = [path['object_name'], path['event_name']]
    fewshot = get_5fewshot_message(base_prompts, limit_words, limit_count)
    message = fewshot + [{"role": "user", "content": prompt}]

    # Generation
    sentences = []
    response = client.chat.completions.create(
                    model="gpt-4o-mini",
                    messages=message,
                    temperature=0.9,
                    top_p=0.9,
                    n=1
                )
    response_sentences = process_mwo_response(response.choices[0].message.content)
    sentences.extend(response_sentences)
    sentences = list(set(sentences)) # Remove duplicates

    # Paraphrase
    paraphrases = []
    chosen_sentence = random.choice(sentences)
    response_paraphrases = paraphrase_mwo(client, chosen_sentence, keywords, 5)
    response_similarities = check_similarity(chosen_sentence, response_paraphrases)
    for para, sim in zip(response_paraphrases, response_similarities):
        if sim > 0.9:
            paraphrases.append(para)
    paraphrases = list(set(paraphrases)) # Remove duplicates

    # Combine
    sentences.extend(paraphrases)
    sentences = list(set(sentences)) # Remove duplicates
    sentences_sample.append(random.choice(sentences))
    
    # Print progress status every 25 paths
    if (i+1) % 25 == 0:
        print(f"Completed {i+1} paths")

# Append to text file
with open('TuringTest/synthetic_generate.txt', 'a', encoding='utf-8') as f:
    for sentence in sentences_sample:
        f.write(f"{sentence}\n")

Completed 25 paths
Completed 50 paths
Completed 75 paths
Completed 100 paths
